In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import gdown
import os
import zipfile

In [ ]:
#Google Drive file URL
file_url = "https://drive.google.com/uc?id=1vHCvd9ZFkY9lfNwUnnMUbDQZjjpby8lv"

#Specify the output file path (the path where you want to save the file)
output_file = "disease.zip"  #Replace with actual file name

#Download the file
gdown.download(file_url, output_file, quiet=False)

zip_file = "disease.zip"  #Name of the downloaded file

#Make sure output directory exists
output_dir = "/content/extracted_file"
os.makedirs(output_dir, exist_ok=True)

#Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

Downloading...
From (original): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F
From (redirected): https://drive.google.com/uc?id=1NfuMblbAumQHfcuJAQCMFA4gR8gf0F8F&confirm=t&uuid=423c28cc-1169-4d1b-a795-6d42e00ee99a
To: /content/disease.zip

  0%|          | 0.00/3.85G [00:00<?, ?B/s]
  1%|          | 22.0M/3.85G [00:00<00:17, 216MB/s]
  1%|          | 47.2M/3.85G [00:00<00:16, 236MB/s]
  2%|▏         | 83.9M/3.85G [00:00<00:12, 295MB/s]
  3%|▎         | 117M/3.85G [00:00<00:12, 309MB/s] 
  4%|▍         | 148M/3.85G [00:00<00:12, 302MB/s]
  5%|▍         | 178M/3.85G [00:00<00:12, 299MB/s]
  5%|▌         | 209M/3.85G [00:00<00:12, 286MB/s]
  6%|▌         | 241M/3.85G [00:00<00:12, 296MB/s]
  7%|▋         | 273M/3.85G [00:00<00:11, 305MB/s]
  8%|▊         | 305M/3.85G [00:01<00:11, 307MB/s]
  9%|▊         | 336M/3.85G [00:01<00:11, 306MB/s]
 10%|▉         | 366M/3.85G [00:01<00:11, 300MB/s]
 10%|█         | 397M/3.85G [00:01<00:11, 302MB/s]
 11%|█         | 428M/3.85G [0

In [ ]:
#Path to main train folder that has many class folders
data_dir = "/content/extracted_file/Retinal Fundus Images - Copy/train"

#Specify the exact classes you want to include
target_classes = ['DR', 'Normal_Fundus']

#Image settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

#Create generators with only selected classes
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,  #This filters folders
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    classes=target_classes,  #This filters folders
    class_mode='sparse',
    subset='validation',
    shuffle=False
)


#Define Model (DenseNet121)

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)


#Compile Model

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


#Train Model

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=1
)


#Save Model

model.save("DR.keras")

Found 19006 images belonging to 2 classes.
Found 2111 images belonging to 2 classes.
594/594 ━━━━━━━━━━━━━━━━━━━━ 498s 548ms/step - accuracy: 0.9920 - loss: 0.0226 - val_accuracy: 1.0000 - val_loss: 7.0297e-05
